In [5]:
import optuna

In [6]:
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [7]:
path="../"

In [8]:
train_org = pd.read_json(path+'ATE_ABSITA_training_set/ate_absita_training.ndjson', lines=True)
train_sin = pd.read_csv(path+'ATE_ABSITA_training_set/sinonimi.csv')
train_emb = pd.read_csv(path+'ATE_ABSITA_training_set/embedding.csv')

dev = pd.concat([pd.read_json(path+'ATE_ABSITA_dev_set/ate_absita_dev.ndjson', lines=True),
                 pd.read_csv(path+'ATE_ABSITA_dev_set/sinonimi.csv'),
                 pd.read_csv(path+'ATE_ABSITA_dev_set/embedding.csv')],
                ignore_index=True)

test_org = pd.read_json(path+'ATE_ABSITA_test_set/ate_absita_gold.ndjson', lines=True)
test_sin = pd.read_csv(path+'ATE_ABSITA_test_set/sinonimi.csv')
test_emb = pd.read_csv(path+'ATE_ABSITA_test_set/embedding.csv')


data_sinonimi = pd.read_csv(path+"ATE_ABSITA_test_set/sinonimi.csv")
data_embedding = pd.read_csv(path+"ATE_ABSITA_test_set/embedding.csv")

In [9]:
train_study_1 = pd.concat([train_org, train_sin, train_emb], ignore_index=True).sample(frac=1)
test_study_1 = pd.concat([test_org, test_sin, test_emb], ignore_index=True).sample(frac=1)

In [10]:
train_study_1.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
dev.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
test_org.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
test_study_1.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
data_sinonimi.drop(columns=['polarities','aspects_position','aspects'], inplace=True)
data_embedding.drop(columns=['polarities','aspects_position','aspects'], inplace=True)

print(f'Contains {len(train_study_1)} sentences')
print(f'Contains {len(dev)} sentences')
print(f'Contains {len(test_study_1)} sentences')
print(f'Contains {len(data_sinonimi)} sentences')
print(f'Contains {len(data_embedding)} sentences')

Contains 9162 sentences
Contains 327 sentences
Contains 3600 sentences
Contains 1200 sentences
Contains 1200 sentences


In [11]:
train_study_1["review_type"] = train_study_1["score"].apply(lambda x: "neg" if x < 5 else "pos")
dev["review_type"] = dev["score"].apply(lambda x: "neg" if x < 5 else "pos")
test_org["review_type"] = test_org["score"].apply(lambda x: "neg" if x < 5 else "pos")
test_study_1["review_type"] = test_study_1["score"].apply(lambda x: "neg" if x < 5 else "pos")
data_sinonimi["review_type"] = data_sinonimi["score"].apply(lambda x: "neg" if x < 5 else "pos")
data_embedding["review_type"] = data_embedding["score"].apply(lambda x: "neg" if x < 5 else "pos")

print(f'TRAIN::\n{train_study_1.review_type.value_counts()}')
print(f'DEV::\n{dev.review_type.value_counts()}')
print(f'TEST::\n{test_study_1.review_type.value_counts()}')
print(f'SINONIMI::\n{data_sinonimi.review_type.value_counts()}')
print(f'EMBEDDING::\n{data_embedding.review_type.value_counts()}')

TRAIN::
pos    6450
neg    2712
Name: review_type, dtype: int64
DEV::
pos    258
neg     69
Name: review_type, dtype: int64
TEST::
pos    2571
neg    1029
Name: review_type, dtype: int64
SINONIMI::
pos    857
neg    343
Name: review_type, dtype: int64
EMBEDDING::
pos    857
neg    343
Name: review_type, dtype: int64


In [12]:
train_study_1.drop(columns=['score'], inplace=True)
dev.drop(columns=['score'], inplace=True)
test_org.drop(columns=['score'], inplace=True)
test_study_1.drop(columns=['score'], inplace=True)
data_sinonimi.drop(columns=['score'], inplace=True)
data_embedding.drop(columns=['score'], inplace=True)

In [13]:
def my_text_to_word_sequence(sentence):
    return keras.preprocessing.text.text_to_word_sequence(sentence,
                                                          filters='!"#$%&()*+,-./:;<=>?@[\\]^_`\'{|}~\t\n',
                                                          lower=True)

# OneHotEncode delle frasi

In [14]:
one_hot_train = OneHotEncoder(sparse=False).fit_transform(
        train_study_1.review_type.to_numpy().reshape(-1, 1))

sentences = [my_text_to_word_sequence(sentence) for sentence in train_study_1['sentence']]

In [15]:
one_hot_dev = OneHotEncoder(sparse=False).fit_transform(
        dev.review_type.to_numpy().reshape(-1, 1))

sentences_dev = [my_text_to_word_sequence(sentence) for sentence in dev['sentence']]

In [16]:
one_hot_test_org = OneHotEncoder(sparse=False).fit_transform(
        test_org.review_type.to_numpy().reshape(-1, 1))

sentences_test_org = [my_text_to_word_sequence(sentence) for sentence in test_org['sentence']]

In [17]:
one_hot_test = OneHotEncoder(sparse=False).fit_transform(
        test_study_1.review_type.to_numpy().reshape(-1, 1))

sentences_test = [my_text_to_word_sequence(sentence) for sentence in test_study_1['sentence']]

In [18]:
one_hot_sin = OneHotEncoder(sparse=False).fit_transform(
        data_sinonimi.review_type.to_numpy().reshape(-1, 1))

sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

In [19]:
one_hot_emb = OneHotEncoder(sparse=False).fit_transform(
        data_embedding.review_type.to_numpy().reshape(-1, 1))

sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

In [20]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_dev):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)


print(f'Il massimo è {max}')

Il massimo è 86


# Embedding delle frasi 

In [21]:
with open(path+"word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

In [22]:
embedded_trainset = np.zeros(shape=(len(sentences), max, 300))
for i, sentence in enumerate(sentences):
    for j, word in enumerate(sentence):
        try:
            embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [23]:
embedded_devset = np.zeros(shape=(len(sentences_dev), max, 300))
for i, sentence in enumerate(sentences_dev):
    for j, word in enumerate(sentence):
        try:
            embedded_devset[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [24]:
embedded_testset_org = np.zeros(shape=(len(sentences_test_org), max, 300))
for i, sentence in enumerate(sentences_test_org):
    for j, word in enumerate(sentence):
        try:
            embedded_testset_org[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [25]:
embedded_testset = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
    for j, word in enumerate(sentence):
        try:
            embedded_testset[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [26]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
    for j, word in enumerate(sentence):
        try:
            embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [27]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
    for j, word in enumerate(sentence):
        try:
            embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

# Model

In [28]:
best_params = optuna.load_study(study_name="ATE", storage="sqlite:///"+path+"optuna_ATE_studio_0.db").best_params

In [29]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params["units"],
                                                             recurrent_dropout=best_params["dropout"],
                                                             activation='tanh')))
model.add(keras.layers.Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.Adam(0.001),
            metrics=['accuracy'])

In [30]:
result = model.fit(embedded_trainset,
                   one_hot_train,
                   validation_data=(embedded_devset, one_hot_dev),
                   epochs=100,
                   batch_size=best_params["batch_size"],
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                            patience=10,
                                                            restore_best_weights=True)])

Epoch 1/100
167/167 [==============================] - 26s 134ms/step - loss: 0.5997 - accuracy: 0.6936 - val_loss: 0.5138 - val_accuracy: 0.7615
Epoch 2/100
167/167 [==============================] - 22s 133ms/step - loss: 0.5121 - accuracy: 0.7405 - val_loss: 0.4255 - val_accuracy: 0.8318
Epoch 3/100
167/167 [==============================] - 22s 132ms/step - loss: 0.4628 - accuracy: 0.7846 - val_loss: 0.4405 - val_accuracy: 0.7920
Epoch 4/100
167/167 [==============================] - 22s 132ms/step - loss: 0.4299 - accuracy: 0.8045 - val_loss: 0.3958 - val_accuracy: 0.8440
Epoch 5/100
167/167 [==============================] - 22s 133ms/step - loss: 0.3768 - accuracy: 0.8357 - val_loss: 0.4338 - val_accuracy: 0.7951
Epoch 6/100
167/167 [==============================] - 22s 132ms/step - loss: 0.3334 - accuracy: 0.8579 - val_loss: 0.5017 - val_accuracy: 0.8043
Epoch 7/100
167/167 [==============================] - 22s 133ms/step - loss: 0.2865 - accuracy: 0.8818 - val_loss: 0.6166 -

In [31]:
model.save_weights('ATE_w_studio3_005.h5')

# EVALUATION

In [32]:
from sklearn.metrics import confusion_matrix, f1_score

## DATASET ORIGINARIO

In [33]:
result_base = model.evaluate(embedded_trainset,one_hot_train,batch_size=best_params['batch_size'])
print(f'DATASET ORIGINARIO{result_base}')

167/167 [==============================] - 5s 27ms/step - loss: 0.3719 - accuracy: 0.8351
DATASET ORIGINARIO[0.371893048286438, 0.8350796699523926]


In [34]:
pred = model.predict(embedded_trainset, batch_size=best_params['batch_size'])
cm = confusion_matrix(one_hot_train.argmax(axis=1), pred.argmax(axis=1))
fscore = f1_score(one_hot_train.argmax(axis=1), pred.argmax(axis=1))

print(f"""Confusion Matrix:
\tP\tN\n
P\t{cm[0][0]}\t{cm[0][1]}\n
N\t{cm[1][0]}\t{cm[1][1]}""")

print(f'\n\nFSCORE:\t{fscore}')

Confusion Matrix:
	P	N

P	1464	1248

N	263	6187


FSCORE:	0.8911775297083183


## TESTSET

In [35]:
result_base=model.evaluate(embedded_testset,one_hot_test,batch_size=best_params['batch_size'])
print(f'DATASET TEST{result_base}')

66/66 [==============================] - 2s 27ms/step - loss: 0.5562 - accuracy: 0.7697
DATASET TEST[0.5561699867248535, 0.7697222232818604]


In [36]:
pred = model.predict(embedded_testset, batch_size=best_params['batch_size'])
cm = confusion_matrix(one_hot_test.argmax(axis=1), pred.argmax(axis=1))
fscore = f1_score(one_hot_test.argmax(axis=1), pred.argmax(axis=1))

print(f"""Confusion Matrix:
\tP\tN\n
P\t{cm[0][0]}\t{cm[0][1]}\n
N\t{cm[1][0]}\t{cm[1][1]}""")

print(f'\n\nFSCORE:\t{fscore}')

Confusion Matrix:
	P	N

P	420	609

N	220	2351


FSCORE:	0.8501175194359067


## TESTSET ORIGINARIO

In [37]:
result_base=model.evaluate(embedded_testset_org,one_hot_test_org,batch_size=best_params['batch_size'])
print(f'DATASET TEST{result_base}')

22/22 [==============================] - 1s 27ms/step - loss: 0.5558 - accuracy: 0.7725
DATASET TEST[0.5558383464813232, 0.7724999785423279]


In [38]:
pred = model.predict(embedded_testset_org, batch_size=best_params['batch_size'])
cm = confusion_matrix(one_hot_test_org.argmax(axis=1), pred.argmax(axis=1))
fscore = f1_score(one_hot_test_org.argmax(axis=1), pred.argmax(axis=1))

print(f"""Confusion Matrix:
\tP\tN\n
P\t{cm[0][0]}\t{cm[0][1]}\n
N\t{cm[1][0]}\t{cm[1][1]}""")

print(f'\n\nFSCORE:\t{fscore}')

Confusion Matrix:
	P	N

P	142	201

N	72	785


FSCORE:	0.8518719479110147


## DATASET SINONIMI

In [39]:
result_base=model.evaluate(embedded_sin,one_hot_sin,batch_size=best_params['batch_size'])
print(f'DATASET SINONIMI{result_base}')

22/22 [==============================] - 1s 27ms/step - loss: 0.5569 - accuracy: 0.7667
DATASET SINONIMI[0.556877613067627, 0.7666666507720947]


In [40]:
pred = model.predict(embedded_sin, batch_size=best_params['batch_size'])
cm = confusion_matrix(one_hot_sin.argmax(axis=1), pred.argmax(axis=1))
fscore = f1_score(one_hot_sin.argmax(axis=1), pred.argmax(axis=1))

print(f"""Confusion Matrix:
\tP\tN\n
P\t{cm[0][0]}\t{cm[0][1]}\n
N\t{cm[1][0]}\t{cm[1][1]}""")

print(f'\n\nFSCORE:\t{fscore}')

Confusion Matrix:
	P	N

P	135	208

N	72	785


FSCORE:	0.8486486486486486


## DATASET EMBEDDING

In [41]:
result_base=model.evaluate(embedded_emb,one_hot_emb,batch_size=best_params['batch_size'])
print(f'DATASET EMBEDDING{result_base}')

22/22 [==============================] - 1s 28ms/step - loss: 0.5558 - accuracy: 0.7700
DATASET EMBEDDING[0.5557941198348999, 0.7699999809265137]


In [42]:
pred = model.predict(embedded_emb, batch_size=best_params['batch_size'])
cm = confusion_matrix(one_hot_emb.argmax(axis=1), pred.argmax(axis=1))
fscore = f1_score(one_hot_emb.argmax(axis=1), pred.argmax(axis=1))

print(f"""Confusion Matrix:
\tP\tN\n
P\t{cm[0][0]}\t{cm[0][1]}\n
N\t{cm[1][0]}\t{cm[1][1]}""")

print(f'\n\nFSCORE:\t{fscore}')

Confusion Matrix:
	P	N

P	143	200

N	76	781


FSCORE:	0.8498367791077257
